In [ ]:
from sys import path

from pandas import DataFrame

path.append("../../../")
from componentes.pipeline.pipeline import Pipeline
from componentes.artefato.arquivo import Arquivo
from componentes.container.container_gold import ContainerGold
from componentes.container.categoria_de_armazenamento import \
    CategoriaDeArmazenamento
from notebooks.silver.paic.dataframe_paic_silver import DataFramePAICSilver

In [ ]:
class FatoResultadoPesuisaPAICSilverParaGold(Pipeline):
    """
    Pipeline de dados responsável por executar o processamento de artefatos
    de categoria silver e gold gerando a fato resultado pesquisa paic
    """
    def __init__(
            self,
            artefato_de_saida: Arquivo,
            container_de_saida: ContainerGold) -> None:
        super().__init__(
            artefato_de_entrada=None,
            container_de_entrada=None,
            artefato_de_saida=artefato_de_saida,
            container_de_saida=container_de_saida)
        self._dimensao_cnae = self._recuperar_dimensao(
            nome_dimensao="dimensao_cnae")
        self._dimensao_variaveis_pesquisa = self._recuperar_dimensao(
            nome_dimensao="dimensao_variaveis_pesquisa")
        self._dimensao_unidade_resultado = self._recuperar_dimensao(
            nome_dimensao="dimensao_unidade_resultado")
        self._dimensao_categoria_empresa = self._recuperar_dimensao(
            nome_dimensao="dimensao_categoria_empresa")

    def _recuperar_dimensao(self, nome_dimensao: str) -> DataFrame:
        """
        Extraí os dados da dimensão especificada da camada gold
        """
        arquivo_dimensao = Arquivo(
            nome=nome_dimensao,
            categoria_de_armazenamento=CategoriaDeArmazenamento.GOLD,
            fonte="dimensao")
        return ContainerGold().extrair_dados(artefato=arquivo_dimensao)

    def _extrair_dados_da_fonte(self) -> DataFrame:
        """
        Extraí os dados usando o objeto DataFramePAICSilver
        """
        self._logger.log_mensagem(mensagem=f"Extraindo a tabela PAIC silver")
        return DataFramePAICSilver().criar_dataframe()

    def _transformar_dados(self, dataframe: DataFrame) -> DataFrame:
        """
        Transforma os dados para criar a modelagem fato
        """
        self._logger.log_mensagem(mensagem="Iniciando transformações")
        dataframe = self._recuperar_chaves_estrangeiras(dataframe=dataframe)
        dataframe = self._selecionar_colunas(dataframe=dataframe)
        dataframe = self._criar_id_resultado(dataframe=dataframe)
        return dataframe

    def _recuperar_chaves_estrangeiras(self, dataframe: DataFrame) -> DataFrame:
        """
        Faz a união com todas as dimensões mapeadas com o intuito de recuperar
        a chave estrangeira
        """
        self._logger.log_mensagem(mensagem=f"Recuperando chaves estrangeiras")
        return (
            dataframe
            .merge(
                right=self._dimensao_cnae,
                left_on="codigo_cnae",
                right_on="codigo",
                how="inner")
            .merge(
                right=self._dimensao_variaveis_pesquisa,
                left_on=["variavel", "ano_pesquisa", "pesquisa"],
                right_on=["variavel", "ano", "pesquisa"],
                how="inner")
            .merge(
                right=self._dimensao_unidade_resultado,
                left_on=["unidade"],
                right_on=["nome"],
                how="inner")
            .merge(
                right=self._dimensao_categoria_empresa,
                left_on=["categoria_empresa"],
                right_on=["nome"],
                how="inner"))

    def _selecionar_colunas(self, dataframe: DataFrame) -> DataFrame:
        """
        Seleciona as colunas necessárias de acordo com a modelagem definida
        """
        self._logger.log_mensagem(mensagem="Selecionando colunas")
        return dataframe[
            ["id_cnae", "id_variavel", "id_unidade", "id_categoria", "valor"]
        ]

    def _criar_id_resultado(self, dataframe: DataFrame) -> DataFrame:
        """
        Criar um novo identificador único para fato
        """
        self._logger.log_mensagem(mensagem="Criando coluna id_resultado")
        dataframe["id_resultado"] = dataframe.reset_index().index + 1
        return dataframe

In [ ]:
artefato_de_saida = Arquivo(
    nome="fato_resultado_pesquisa_paic",
    categoria_de_armazenamento=CategoriaDeArmazenamento.GOLD,
    fonte="fato")
container_de_saida = ContainerGold()
pipeline = FatoResultadoPesuisaPAICSilverParaGold(
    artefato_de_saida=artefato_de_saida,
    container_de_saida=container_de_saida)
pipeline.executar_etl()